In [1]:
# Import required packages
from os import path 
import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
from torch import nn, Tensor
from torch.optim import AdamW
from tqdm import tqdm, trange
import zuko
from zuko.flows import Distribution, NSF
from zuko.distributions import DiagNormal, BoxUniform, Minimum
from zuko.flows import DistributionModule, FlowModule, Unconditional
from hnne import HNNE

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

In [2]:
panda = Robot(verbose=False)
# data generation
X, y = load_data(robot=panda)
# build dimension reduction model
hnne, ds, loader = get_hnne_model(X, y)
# Build Generative model, NSF
# Neural spline flow (NSF) with 3 sample features and 5 context features
flow, optimizer, scheduler = get_flow_model()

hnne load successfully from ./weights/panda/hnne.pickle
Model load successfully from ./weights/panda/nsf.pth


In [3]:
mu = torch.randn(size=(7,))
mu

tensor([ 1.6594, -0.6866, -0.4550, -0.0094,  0.2435, -0.0994, -0.0403])

In [4]:
iflow = get_iflow_model(flow, init_sample=mu, shrink_ratio=0)
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=loader, 
                      model=iflow, 
                      num_data=1, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,1.000000,1.00
mean,0.000682,0.23
std,NaN,NaN
min,0.000682,0.23
25%,0.000682,0.23
50%,0.000682,0.23
75%,0.000682,0.23
max,0.000682,0.23


In [5]:
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=loader, 
                      model=flow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.007491,0.061400
std,0.005600,0.004269
min,0.003665,0.060000
25%,0.005048,0.060000
50%,0.005811,0.060000
75%,0.007762,0.060000
max,0.045556,0.090000


In [6]:
nflow = get_nflow_model(flow=flow)
df, err = test_l2_err(config, 
                      robot=panda, 
                      loader=loader, 
                      model=nflow, 
                      num_data=100, 
                      num_samples=1000, 
                      inference=True)
# ax1 = df.plot.scatter(x='log_prob', y='l2_err')
df.describe()

,l2_err,time_diff(per 1000)
count,100.000000,100.000000
mean,0.005104,0.061700
std,0.003237,0.004034
min,0.002405,0.060000
25%,0.003682,0.060000
50%,0.004626,0.060000
75%,0.005526,0.060000
max,0.031924,0.080000


In [7]:
traj_dir = sample_ee_traj(robot=panda, load_time='')

100%|██████████| 20/20 [00:00<00:00, 11908.87it/s]

mkdir ./data/panda/trajectory/05270002
./data/panda/trajectory/05270002/ load successfully.


In [17]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=nflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

[[ 0.15556125 -0.153328    0.93654864  0.54758967]
 [-0.31405448  0.11573337  0.05309009 -0.26536473]
 [ 0.15558422 -0.15335853  0.93653168  0.54757972]]
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.015611 -13.576136     109.203966
std     0.040748   1.766252      59.970886
min     0.000894 -14.990849      14.934485
25%     0.002558 -14.512943      58.277267
50%     0.004321 -14.093352      96.973089
75%     0.005977 -13.615041     154.681406
max     0.180599  -7.551800     253.159541
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000      20.000000
mean    0.003897 -14.639510      42.840789
std     0.001393   0.596778      30.498264
min     0.002122 -15.776011       1.308722
25%     0.002996 -15.025955      20.224026
50%     0.003446 -14.684079      35.578676
75%     0.004612 -14.271796      61.051255
max     0.007517 -13.456232     131.438494
          l2_err   log_prob  ang_errs(sum)
count  20.000000  20.000000  

In [18]:
generate_traj_via_model(hnne=hnne, num_traj=3, model=iflow, robot=panda, traj_dir=traj_dir, enable_regenerate=True)

[[ 0.15621272 -0.15448435  0.93693022  0.54855924]
 [-0.51362493 -0.15450349  0.31226371 -0.43338048]
 [-0.31405745  0.11573153  0.05308288 -0.265363  ]]
          l2_err  log_prob  ang_errs(sum)
count  20.000000       0.0      20.000000
mean    0.003677       NaN      19.219002
std     0.002013       NaN      18.285670
min     0.001140       NaN       0.168251
25%     0.002270       NaN       1.849405
50%     0.003396       NaN      15.588828
75%     0.004468       NaN      29.328557
max     0.008090       NaN      63.307452
          l2_err  log_prob  ang_errs(sum)
count  20.000000       0.0      20.000000
mean    0.003840       NaN      10.774111
std     0.002419       NaN      10.340183
min     0.000780       NaN       0.017562
25%     0.001443       NaN       2.832153
50%     0.004068       NaN       9.472603
75%     0.006057       NaN      13.851911
max     0.007734       NaN      36.032974
          l2_err  log_prob  ang_errs(sum)
count  20.000000       0.0      20.000000
mean  